<a href="https://colab.research.google.com/github/HJbgk12/2.-AI-Goorm-QA-model/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/KoreanMRC

/content/drive/MyDrive/KoreanMRC


In [ ]:
! apt-get install -y g++ openjdk-8-jdk python3-dev curl git build-essential
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
!pip install transformers

In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [ ]:
!pip install -r requirements.txt
!pip install .

In [ ]:
import torch
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import BertConfig
from tqdm import tqdm

#from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model
#from KoBERT.kobert_hf.kobert_tokenizer import KoBERTTokenizer

import ko_mrc.datasets as datasets
import ko_mrc.utils as utils
import ko_mrc.models as models

In [ ]:
dataset = datasets.KoMRC.load('data/train3.json')
print("Number of Questions", len(dataset))
print(dataset[0])

Number of Questions 255462
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': [{'text': '한 달가량', 'answer_start': 478}, {'text': '한 달', 'answer_start': 478}]}


In [ ]:
dataset = datasets.TokenizedKoMRC.load('data/train3.json')
train_dataset, eval_dataset = datasets.TokenizedKoMRC.split(dataset)
print("Number of Questions", len(train_dataset), len(eval_dataset))
print(train_dataset[0])

Number of Questions 229916 25546
{'guid': '86cf7f16da224f81a7a86b61ae2d539f', 'context_original': '일본에서는 현재 교사의 40% 이상이 50대인데요. 경험 많은 교사들이 조만간 대규모로 퇴직을 하게 돼 지자체마다 우수한 교사를 확보하기 위한 쟁탈전을 벌이고 있습니다. [리포트] 지난해 12월 후쿠오카 현 교육위원회가 도쿄에서 실시한 채용시험입니다. 대상은 현직 교사. 응시자의 90%가 합격해 올봄부터 후쿠오카에서 근무하게 됐습니다. 교사 수가 많은 도쿄 등 수도권에서 이른바 스카우트를 하는 것입니다. 부모 간병이나 육아 등을 위해 고향으로 돌아가려는 사람들이 지원한다고 합니다. 전국 교사 채용 수는 1982년을 정점으로 계속 줄어 2000년에는 5분의 1까지 줄었는데요. 당시 퇴직자 수가 적고 저출산으로 교사를 늘릴 필요가 없었다는 게 이유로 지적됩니다. 경력 교사를 빼앗기는 지역에서는 고민이 많습니다.[가나가와현 교육위원회 : 가장 활약할 나이의 교사가 빠져나가면 학교 운영에 어려움이 생깁니다.] 전국의 교사 인원은 법률로 정해져 있어 지자체 차원에서는 근본적인 해결이 어려운데요. 교사 쟁탈전이 가열되면 지역에 따라 교육의 질에 편차가 생길 수 있다고 우려하고 있습니다.', 'context_position': [(0, 2), (2, 4), (4, 5), (6, 8), (9, 11), (11, 12), (13, 15), (15, 16), (17, 19), (19, 20), (21, 23), (23, 24), (24, 27), (27, 28), (29, 31), (32, 33), (33, 34), (35, 37), (37, 38), (38, 39), (40, 43), (44, 45), (45, 47), (47, 48), (49, 51), (51, 52), (53, 54), (54, 55), (56, 57), (58, 61), (61, 63), (64, 66), (66, 67), (68

In [ ]:
sample = train_dataset[0]
print(sample['context'][sample['answers'][0]['start']:sample['answers'][0]['end']+1])

['우수', '한', '교사', '를']


In [ ]:
tokenizer = utils.Tokenizer.build_vocab(dataset)
print(tokenizer.sample2ids(train_dataset[0]))

Counting Vocab: 100%|██████████| 255462/255462 [10:12<00:00, 417.22it/s]

{'guid': '86cf7f16da224f81a7a86b61ae2d539f', 'context': '일본에서는 현재 교사의 40% 이상이 50대인데요. 경험 많은 교사들이 조만간 대규모로 퇴직을 하게 돼 지자체마다 우수한 교사를 확보하기 위한 쟁탈전을 벌이고 있습니다. [리포트] 지난해 12월 후쿠오카 현 교육위원회가 도쿄에서 실시한 채용시험입니다. 대상은 현직 교사. 응시자의 90%가 합격해 올봄부터 후쿠오카에서 근무하게 됐습니다. 교사 수가 많은 도쿄 등 수도권에서 이른바 스카우트를 하는 것입니다. 부모 간병이나 육아 등을 위해 고향으로 돌아가려는 사람들이 지원한다고 합니다. 전국 교사 채용 수는 1982년을 정점으로 계속 줄어 2000년에는 5분의 1까지 줄었는데요. 당시 퇴직자 수가 적고 저출산으로 교사를 늘릴 필요가 없었다는 게 이유로 지적됩니다. 경력 교사를 빼앗기는 지역에서는 고민이 많습니다.[가나가와현 교육위원회 : 가장 활약할 나이의 교사가 빠져나가면 학교 운영에 어려움이 생깁니다.] 전국의 교사 인원은 법률로 정해져 있어 지자체 차원에서는 근본적인 해결이 어려운데요. 교사 쟁탈전이 가열되면 지역에 따라 교육의 질에 편차가 생길 수 있다고 우려하고 있습니다.', 'question': '일본지자체들이 뭘 확보하기 위해서 쟁탈전을 벌이고 있어?', 'position': [(0, 2), (2, 4), (4, 5), (6, 8), (9, 11), (11, 12), (13, 15), (15, 16), (17, 19), (19, 20), (21, 23), (23, 24), (24, 27), (27, 28), (29, 31), (32, 33), (33, 34), (35, 37), (37, 38), (38, 39), (40, 43), (44, 45), (45, 47), (47, 48), (49, 51), (51, 52), (53, 54), (54, 55), (56, 57), (58, 61), (61, 63), (64, 66), (66, 67), (68, 7

In [ ]:
train_dataset = utils.TokenizerWrapperDataset(train_dataset, tokenizer)
eval_dataset = utils.TokenizerWrapperDataset(eval_dataset, tokenizer)

print(train_dataset[0])

{'guid': '86cf7f16da224f81a7a86b61ae2d539f', 'context': '일본에서는 현재 교사의 40% 이상이 50대인데요. 경험 많은 교사들이 조만간 대규모로 퇴직을 하게 돼 지자체마다 우수한 교사를 확보하기 위한 쟁탈전을 벌이고 있습니다. [리포트] 지난해 12월 후쿠오카 현 교육위원회가 도쿄에서 실시한 채용시험입니다. 대상은 현직 교사. 응시자의 90%가 합격해 올봄부터 후쿠오카에서 근무하게 됐습니다. 교사 수가 많은 도쿄 등 수도권에서 이른바 스카우트를 하는 것입니다. 부모 간병이나 육아 등을 위해 고향으로 돌아가려는 사람들이 지원한다고 합니다. 전국 교사 채용 수는 1982년을 정점으로 계속 줄어 2000년에는 5분의 1까지 줄었는데요. 당시 퇴직자 수가 적고 저출산으로 교사를 늘릴 필요가 없었다는 게 이유로 지적됩니다. 경력 교사를 빼앗기는 지역에서는 고민이 많습니다.[가나가와현 교육위원회 : 가장 활약할 나이의 교사가 빠져나가면 학교 운영에 어려움이 생깁니다.] 전국의 교사 인원은 법률로 정해져 있어 지자체 차원에서는 근본적인 해결이 어려운데요. 교사 쟁탈전이 가열되면 지역에 따라 교육의 질에 편차가 생길 수 있다고 우려하고 있습니다.', 'question': '일본지자체들이 뭘 확보하기 위해서 쟁탈전을 벌이고 있어?', 'position': [(0, 2), (2, 4), (4, 5), (6, 8), (9, 11), (11, 12), (13, 15), (15, 16), (17, 19), (19, 20), (21, 23), (23, 24), (24, 27), (27, 28), (29, 31), (32, 33), (33, 34), (35, 37), (37, 38), (38, 39), (40, 43), (44, 45), (45, 47), (47, 48), (49, 51), (51, 52), (53, 54), (54, 55), (56, 57), (58, 61), (61, 63), (64, 66), (66, 67), (68, 7

In [ ]:
collator = utils.Collator(tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collator, num_workers=1)
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False, collate_fn=collator, num_workers=1)
batch = next(iter(train_loader))
print(batch['input_ids'].shape)
print(batch['input_ids'])
print(list(batch.keys()))

torch.Size([8, 706])
tensor([[    2,  1484,  1664,  ...,     0,     0,     0],
        [    2,  4078, 35468,  ...,     0,     0,     0],
        [    2, 31299, 31300,  ...,     0,     0,     0],
        ...,
        [    2, 43157,    40,  ...,     0,     0,     0],
        [    2,   798,     7,  ...,     0,     0,     0],
        [    2, 13054,  9884,  ...,    15,  3452,     3]])
['guid', 'context', 'question', 'position', 'input_ids', 'token_type_ids', 'start', 'end', 'attention_mask']


In [ ]:
config = BertConfig(
     vocab_size=tokenizer.vocab_size,
     max_position_embeddings=1024,
     hidden_size=256,
     num_hidden_layers=4,
     num_attention_heads=4,
     intermediate_size=1024
)
model = models.BertForQuestionAnswering(config)
model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(122633, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
import os

In [ ]:
writer = SummaryWriter(log_dir='review_train4')
os.makedirs('dump8', exist_ok=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
step = 0

In [ ]:
for epoch in range(1, 31):
    print("Epoch", epoch)
    for batch in tqdm(train_loader, desc='Train'):
        del batch['guid'], batch['context'], batch['question'], batch['position']
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start')
        end = batch.pop('end')
        
        start_logits, end_logits = model(**batch)
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()

        step += 1
        writer.add_scalar('Train Loss', loss.item(), step)
        del batch, start, end, start_logits, end_logits, loss

    with torch.no_grad():
        losses = []
        for batch in tqdm(eval_loader, desc="Evaluation"):
            del batch['guid'], batch['context'], batch['question'], batch['position']
            batch = {key: value.cuda() for key, value in batch.items()}
            start = batch.pop('start')
            end = batch.pop('end')

            start_logits, end_logits = model(**batch)
            loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

            losses.append(loss.item())
            del batch, start, end, start_logits, end_logits, loss
        loss = sum(losses) / len(losses)
        writer.add_scalar('Eval Loss', loss, step)

    model.save_pretrained(f'dump8/model.{epoch}')
        

In [ ]:
model = models.BertForQuestionAnswering.from_pretrained('dump8/model.30')
model.cuda()
model.eval() 

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(117152, 256, padding_idx=0)
      (position_embeddings): Embedding(1024, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwis

텐서보드

In [ ]:
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
log_dir = "log/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir reviewAIhub/

In [ ]:
for idx, raw_sample in zip(range(1, 4), train_dataset):
    sample = dict(raw_sample) 
    context = sample.pop('context')
    question = sample.pop('question')
    position = sample.pop('position')
    start, end = sample.pop('start'), sample.pop('end')
    del sample['guid']

    sample = {key: value.cuda()[None, :] for key, value in sample.items()}
    
    with torch.no_grad():
        start_logits, end_logits = model(**sample)
    start_logits.squeeze_(0), end_logits.squeeze_(0)

    print(f'------{idx}------')
    print('Context:', context)
    print('Question:', question)
    print('Answer:', tokenizer.logits2answer(raw_sample, start_logits, end_logits))

------1------
Context: 현대사진의 선각자로 평가받는 로버트 프랭크, ‘소나무 작가’ 배병우, 일본 작가 히로시 스키모토, 인물·다큐멘터리 사진작가 스티브 매커리, 몽상적인 내용을 사진에 담는 베르나르 포콩, 벌거벗은 젊은이들 사진으로 유명한 라이언 맥긴리…. 늦가을 화랑가에 국내외 유명 사진작가의 작품전이 줄을 잇고 있다. ‘잡화점식’ 전시에서 벗어나 각기 다른 테마를 다룬 기획전이어서 더욱 주목된다. 디지털 문화의 확산으로 사진과 그림의 경계가 허물어지는 경향도 확인할 수 있다. ○히로시·프랭크 등 거장 총출동도쿄와 뉴욕, 유럽을 주 무대로 활동하는 ‘밀리언달러’ 사진작가 히로시(66)의 개인전은 내달 5일 서울 한남동 삼성미술관 리움에서 개막한다. 히로시는 시간과 공간이라는 주제와 흑백사진의 표현을 연구해온 작가. 특정한 순간을 포착하는 대신 한 개의 프레임에 전체 작품을 찍어서 시간의 흐름을 연결한 게 특징이다. 이번 전시에는 ‘극장’ ‘바다풍경’ ‘초상’ 시리즈 등 100여점을 내보인다.스위스 출신 사진작가 프랭크(89)의 작품전은 서울 방이동 한미사진미술관에서 오는 9일부터 내년 2월9일까지 이어진다. 1958년작 ‘미국인’을 비롯해 1970~90년대 폴라로이드 작품 등 프랭크의 70년 사진 인생을 한눈에 보여주는 오리지널 프린트 115점이 걸린다. 7일부터 서울 통의동 대림미술관에서 열리는 ‘청춘, 그 찬란한 기록’전은 미국 유망 사진작가 라이언 맥긴리(25)의 국내 첫 개인전이다. 자유와 불안, 방황, 일탈, 열정 등 젊은이들의 내면에 공존하는 감정을 렌즈로 솔직하게 잡아낸 대표작 100여점을 만날 수 있다. 맥긴리는 25세의 젊은 나이에 미국 휘트니미술관과 뉴욕현대미술관에서 개인전을 열어 세계 사진계의 주목을 받고 있다. 프랑스 사진작가 포콩의 몽상적인 내용을 담은 신작들은 오는 9일부터 부산 고은사진미술관에 걸린다. 유년시절 체험한 사랑과 열정, 외로움 등을 푸른색으로 녹여낸 작품이 눈길을 끈다. 이 밖에 필립 할스만(세종문화

/content/drive/My Drive/KoreanMRC/ko_mrc/utils.py:104: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start = index // len(end_prob)


In [ ]:
test_dataset = datasets.TokenizedKoMRC.load('data/test.json')
test_dataset = utils.TokenizerWrapperDataset(test_dataset, tokenizer)
print("Number of Questions", len(test_dataset))
print(test_dataset[0])

In [ ]:
rows = []
for raw_sample in tqdm(test_dataset, "Testing"):
        sample = dict(raw_sample) 
        guid = sample.pop('guid')
        context = sample.pop('context')
        position = sample.pop('position')
        start, end = sample.pop('start'), sample.pop('end')
        del sample['question']

        sample = {key: value.cuda()[None, :] for key, value in sample.items()}
        start_logits, end_logits = model(**sample)
        start_logits.squeeze_(0), end_logits.squeeze_(0)
    
        rows.append([guid, tokenizer.logits2answer(raw_sample, start_logits, end_logits)])
        

In [ ]:
import os
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/submission.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for raw_sample in tqdm(test_dataset, "Testing"):
        sample = dict(raw_sample) 
        guid = sample.pop('guid')
        context = sample.pop('context')
        position = sample.pop('position')
        start, end = sample.pop('start'), sample.pop('end')
        del sample['question']

        sample = {key: value.cuda()[None, :] for key, value in sample.items()}
        start_logits, end_logits = model(**sample)
        start_logits.squeeze_(0), end_logits.squeeze_(0)

        rows.append([guid, tokenizer.logits2answer(raw_sample, start_logits, end_logits)])
    
    writer.writerows(rows)